# select 语句

select 语句选择可能发生通信的集。它看起来与 switch 语句类似，但其 case 为涉及到通信的操作。

    select {
    case 发送语句或接收语句:
            ...
    case ...:
            ...
            ...
    default: 
            ...
    }
    
default 语句是可选的，可以在 select 语句中任何位置；没有 fallthrough 语句。在任何一个 case 中执行 break 或者 return，select 就结束了。

select 做的就是：选择处理列出的多个通信情况中的一个。

- 如果都阻塞了，会等待直到其中一个可以处理
- 如果多个可以处理，随机选择一个
- 如果没有通道操作可以处理并且写了 default 语句，它就会执行：default 永远是可运行的（这就是准备好了，可以执行）。

在 select 中使用发送操作并且有 default 可以确保发送不被阻塞！如果没有 default，select 就会一直阻塞。

In [16]:
import "fmt"

c1 := make(chan string, 1)
c2 := make(chan string, 1)

c1 <- "hello"
c2 <- "How are you"

select {
case msg1 := <-c1:
    fmt.Println("c1 received: ", msg1)
case msg2 := <-c2:
    fmt.Println("c2 received: ", msg2)
default:
    fmt.Println("No data received.")
}

c2 received:  How are you


In [3]:
import (
    "fmt"
    "time"
)


func pump1(ch chan int) {
    for i := 0; i < 10; i++ {
        ch <- i * 2
    }
}

func pump2(ch chan int) {
    for i := 0; i < 10; i++ {
        ch <- i + 5
    }
}

func suck(ch1, ch2 chan int) {
    for {
        select {
        case v := <-ch1:
            fmt.Printf("Received on channel 1: %d\n", v)
        case v := <-ch2:
            fmt.Printf("Received on channel 2: %d\n", v)
        }
    }
}

ch1 := make(chan int)
ch2 := make(chan int)

go pump1(ch1)
go pump2(ch2)
go suck(ch1, ch2)

time.Sleep(1e9)

Received on channel 2: 5
Received on channel 1: 0
Received on channel 2: 6
Received on channel 2: 7
Received on channel 2: 8
Received on channel 1: 2
Received on channel 2: 9
Received on channel 2: 10
Received on channel 2: 11
Received on channel 1: 4
Received on channel 2: 12
Received on channel 2: 13
Received on channel 1: 6
Received on channel 2: 14
Received on channel 1: 8
Received on channel 1: 10
Received on channel 1: 12
Received on channel 1: 14
Received on channel 1: 16
Received on channel 1: 18


In [ ]:
import (
    "fmt"
    "time"
)

func write(ch chan int) {
    for i := 0; i < 10; i++ {
        select {
        case ch <- int(i):
            fmt.Println("write", i)
        default:
            fmt.Println("channel full")
        }
        time.Sleep(time.Millisecond * 500)
    }
}

output1 := make(chan int, 5)
go write(output1)
for s := range output1 {
    fmt.Println("res:", s)
    time.Sleep(2 * time.Second)
}

write 0
res: 0
write 1
write 2
write 3
res: 1
write 4
write 5
write 6
channel full
res: 2
write 8
channel full
res: 3
res: 4
res: 5
res: 6
res: 8
